In [14]:
import os
import getpass

os.environ["LANGCHAIN_API_KEY"]=getpass.getpass()
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["FIREWORKS_API_KEY"]=getpass.getpass()

 ········
 ········


In [15]:
from langchain_fireworks import ChatFireworks
model=ChatFireworks(model="accounts/fireworks/models/mixtral-8x7b-instruct")

# Data ingestion

## Text document

In [10]:
from langchain_community.document_loaders import TextLoader
loader_text=TextLoader("speech.txt")

In [11]:
text_docucment=loader_text.load()
text_docucment

[Document(page_content='I have a dream that one day down in Alabama, with its vicious racists, with its governor having his lips dripping with the words of interposition and nullification â€“ one day right there in Alabama little black boys and black girls will be able to join hands with little white boys and white girls as sisters and brothers.\n\nI have a dream today.\n\nI have a dream that one day every valley shall be exalted, and every hill and mountain shall be made low, the rough places will be made plain, and the crooked places will be made straight, and the glory of the Lord shall be revealed and all flesh shall see it together.\n\nThis is our hope. This is the faith that I go back to the South with. With this faith we will be able to hew out of the mountain of despair a stone of hope. With this faith we will be able to transform the jangling discords of our nation into a beautiful symphony of brotherhood. With this faith we will be able to work together, to pray together, to 

## Web based

In [18]:
# pip install bs4

In [19]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

In [22]:
loader_web=WebBaseLoader(web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
                        bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("post-content", "post-title", "post-header"))))

In [24]:
web_document=loader_web.load()
# web_document

## PDF based

In [28]:
# pip install pypdf

In [29]:
from langchain_community.document_loaders import PyPDFLoader

In [30]:
loader_pdf=PyPDFLoader("attention.pdf")

In [31]:
pdf_document=loader_pdf.load()

In [33]:
# pdf_document

# Transform

In [34]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=splitter.split_documents(pdf_document)

In [35]:
documents[:5]

[Document(page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing 

# Embed

In [47]:
# pip install langchain_chroma

In [43]:
# pip install -qU langchain-openai

In [50]:
from langchain_fireworks import FireworksEmbeddings
from langchain_chroma import Chroma
db=Chroma.from_documents(documents[:20],FireworksEmbeddings())

In [54]:
## vector database
query="What is attention is all you need research paper?"
result=db.similarity_search(query)
result[0].page_content

'3.2 Attention\nAn attention function can be described as mapping a query and a set of key-value pairs to an output,\nwhere the query, keys, values, and output are all vectors. The output is computed as a weighted sum\n3'

# Retriver and chain

In [55]:
from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_template(""" Answer the following qustions based only on the context provided.
                                            Think step by step before ansering the question.
                                            <context>
                                            {context}
                                            </context>
                                            Question: {input}""")

In [56]:
#create stuff document chain

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(model,prompt)

In [57]:
retriever=db.as_retriever()

In [58]:
retriever

VectorStoreRetriever(tags=['Chroma', 'FireworksEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x0000016F17D07A60>)

In [59]:
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [61]:
response=retrieval_chain.invoke({"input":"what is attention function"})

In [62]:
response["answer"]

'Based on the provided context, an attention function is a type of mapping that takes a query vector and a set of key-value pairs, all of which are vectors, and produces an output vector. The keys and values are packed together into matrices K and V, respectively. The output is computed as a weighted sum of the values, where the weights are determined by the compatibility of the query with the keys, as given by the softmax function applied to the product of the query and keys matrices, scaled by the square root of the dimension of the keys, dk. The two most commonly used attention functions are dot-product (multiplicative) attention and additive attention. Dot-product attention is identical to the algorithm described in the context, except for a scaling factor of 1/√dk, while additive attention computes the compatibility function using a feed-forward network with a single hidden layer. Dot-product attention is faster and more space-efficient in practice, but additive attention may outp